<a href="https://colab.research.google.com/github/totminaekaterina/RUSSE-2022-Detoxification/blob/main/delete.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install ufal.udpipe

     |████████████████████████████████| 304 kB 5.5 MB/s 
  Created wheel for ufal.udpipe: filename=ufal.udpipe-1.2.0.3-cp37-cp37m-linux_x86_64.whl size=5626623 sha256=6e3ba69165f794f46625efaf4476344d7df6cb47af26b031635d2818715db602
  Stored in directory: /root/.cache/pip/wheels/b8/b5/8e/3da091629a21ce2d10bf90759d0cb034ba10a5cf7a01e83d64
Successfully built ufal.udpipe


In [2]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import random

import gensim
from ufal.udpipe import Model, Pipeline

In [3]:
!git clone https://github.com/skoltech-nlp/russe_detox_2022

Cloning into 'russe_detox_2022'...
remote: Enumerating objects: 82, done.
remote: Counting objects: 100% (53/53), done.
remote: Compressing objects: 100% (35/35), done.
remote: Total 82 (delta 22), reused 43 (delta 16), pack-reused 29
Unpacking objects: 100% (82/82), done.


In [4]:
!wget https://raw.githubusercontent.com/totminaekaterina/RUSSE-2022-Detoxification/main/prepared_data/dev_answers.csv
!wget https://raw.githubusercontent.com/totminaekaterina/RUSSE-2022-Detoxification/main/prepared_data/test_answers.csv

--2022-02-13 15:08:28--  https://raw.githubusercontent.com/totminaekaterina/RUSSE-2022-Detoxification/main/prepared_data/dev_answers.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 81930 (80K) [text/plain]
Saving to: ‘dev_answers.csv’

dev_answers.csv     100%[===================>]  80.01K  --.-KB/s    in 0.01s   

2022-02-13 15:08:28 (6.12 MB/s) - ‘dev_answers.csv’ saved [81930/81930]

--2022-02-13 15:08:28--  https://raw.githubusercontent.com/totminaekaterina/RUSSE-2022-Detoxification/main/prepared_data/test_answers.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.109.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP reques

In [5]:
with open('/content/russe_detox_2022/baselines/delete/toxic_vocab_extended.txt', 'r') as file:
    toxic_words = file.readlines()
toxic_words = [sentence.strip() for sentence in toxic_words]

In [6]:
modelfile = '/content/russe_detox_2022/baselines/delete/udpipe_syntagrus.model'
model_udpipe = Model.load(modelfile)
process_pipeline = Pipeline(model_udpipe, 'tokenize', Pipeline.DEFAULT, Pipeline.DEFAULT, 'conllu')

In [7]:
def tokenize(text, tags=False, lemmas=False):
    processed = process_pipeline.process(text)
    content = [l for l in processed.split('\n') if not l.startswith('#')]
    tagged = [w.split('\t') for w in content if w]
    
    tokens = []
    for token in tagged:
        if token[3] == 'PUNCT':
            continue
        
        token_res = ''
        if lemmas:
            token_res = token[2]
        else:
            token_res = token[1]
        if tags:
            token_res += '_' + token[3]
        tokens.append(token_res)
        
    return tokens

In [8]:
df = pd.read_csv('dev_answers.csv', sep='\t')
df.head()

,toxic_comment
0,Теплые температуры горения хватит чтобы её рас...
1,а ты что там был.ты вообще служил
2,а сам где кормишься?
3,"этому сайту я давно не доверяю, пишут разную е..."
4,у меня сегодня подобный звонок был. достали


In [9]:
toxic_inputs_dev = df['toxic_comment'].tolist()

In [10]:
ds = pd.read_csv('test_answers.csv', sep='\t')
ds.head()

,toxic_comment
0,Кто придумывает эту историю
1,"В такой ситуации виноваты люди из Ростелекома,..."
2,"Актёр может и не плохой,но как человек - не хо..."
3,наказывайте всех кто нарушает общественный пор...
4,Такие же люди и привели этих людей


In [11]:
toxic_inputs_test = ds['toxic_comment'].tolist()

In [12]:
results_dev = []

for sample in tqdm(toxic_inputs_dev):
    try:
        tokens_lemmas = tokenize(sample, lemmas=True)
    except:
        print(sample)
        tokens_lemmas = sample.split(' ')
    tokens = tokenize(sample, lemmas=False)
    cleaned_sentence = [tokens[i] for i, word in enumerate(tokens_lemmas) if word not in toxic_words]
        
    results_dev.append(' '.join(cleaned_sentence))

100%|██████████| 800/800 [00:52<00:00, 15.22it/s]


In [13]:
pd.DataFrame({"answer": results_dev})["answer"].apply(pd.Series).to_csv("/content/dev_cleaned.csv", index=False)

In [14]:
results_test = []

for sample in tqdm(toxic_inputs_test):
    try:
        tokens_lemmas = tokenize(sample, lemmas=True)
    except:
        print(sample)
        tokens_lemmas = sample.split(' ')
    tokens = tokenize(sample, lemmas=False)
    cleaned_sentence = [tokens[i] for i, word in enumerate(tokens_lemmas) if word not in toxic_words]
        
    results_test.append(' '.join(cleaned_sentence))

100%|██████████| 875/875 [00:59<00:00, 14.77it/s]


In [15]:
pd.DataFrame({"answer": results_test})["answer"].apply(pd.Series).to_csv("/content/test_cleaned.csv", index=False)